# BackTesting 


In [15]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-USD.csv', parse_dates=['Date'], index_col=['Date'])
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-12-13,100046.648438,101888.804688,99233.281250,101459.257812,101459.257812,56894751583
2024-12-14,101451.437500,102618.882812,100634.054688,101372.968750,101372.968750,40422968793
2024-12-15,101373.531250,105047.539062,101227.031250,104298.695312,104298.695312,51145914137


In [24]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, plot_heatmaps
import talib 


class SMAcross(Strategy):
    def init(self):
        self.sma = self.I(talib.SMA, self.data.Close, 50)
    def next(self):
        price = self.data.Close[-1]

        if price > self.sma[-1] and not self.position.is_long:
            self.buy()
        elif price < self.sma[-1] and self.position.is_long:
            self.position.close()
bt = Backtest(data, SMAcross)
bt.run()

C:\Users\Ivo\AppData\Local\Temp\ipykernel_20128\2119554169.py:16: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SMAcross)


Start                     2014-09-17 00:00:00
End                       2024-12-17 00:00:00
Duration                   3744 days 00:00:00
Exposure Time [%]                   59.225634
Equity Final [$]               6083860.522354
Equity Peak [$]                6083860.522354
Return [%]                       60738.605224
Buy & Hold Return [%]            23336.641685
Return (Ann.) [%]                   86.790702
Volatility (Ann.) [%]               97.814186
Sharpe Ratio                         0.887302
Sortino Ratio                        2.756828
Calmar Ratio                         1.520234
Max. Drawdown [%]                  -57.090346
Avg. Drawdown [%]                   -8.805823
Max. Drawdown Duration      755 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   89
Win Rate [%]                        29.213483
Best Trade [%]                      414.20235
Worst Trade [%]                    -10.558727
Avg. Trade [%]                    

In [48]:
bt.plot()

ValueError: failed to validate DatetimeTickFormatter(id='p1706', ...).days: expected a value of type str, got ['%d %b', '%a %d'] of type list